In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from multiprocessing import Pool
from Bio import SeqIO
from Bio import motifs
from Bio.Seq import Seq
import os 

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/ubuntu/workspace/AMPsDetector/src')

from pssm import *

In [3]:
from os import getpid
from functools import partial

num_partitions = 24 #number of partitions to split dataframe 22
num_cores = 24 #number of cores on your machine 22

def convert_pssm(df):
    process_id =str(getpid())
    path = "/mnt/vdb/Bat/pws/transpi/reps/"
    ## path = "/mnt/vdb/Bat/pws/cd100/reps"
    pro_plk= path + "chunk"+process_id+".pkl"
    if not os.path.exists(path):
        os.makedirs(path)
    
    print(process_id+" Start")

    char_code_list = [char for char in "ACDEFGHIKLMNPQRSTVWY"] 

    ready_df = pd.DataFrame(columns = ['ID', 'Sequence','reps', 'length'])
    for index, seq_record in df.iterrows():
        #print(seq_record)
        m = motifs.create([ seq_record["Sequence"] ] , alphabet="ACDEFGHIKLMNPQRSTVWY")  
        index_pattern = [char for char in m.consensus]
        file_name = '/mnt/vdb/Bat/pws/transpi/images/'+seq_record["ID"]+'.png'
        ## file_name = '/mnt/vdb/Bat/pws/cd100/images/'+seq_record["ID"]+'.png'
        pssm = get_pssm(m)
        result_df = convert_to20X20_(pssm , index_pattern)
        result_df = scale_by(result_df/len(m.consensus))
        create_image(result_df,file_name) 

        _flat = result_df.values.flatten()
 
        ready_df = ready_df.append({'ID': seq_record["ID"], 'Sequence': seq_record["Sequence"],
                                   'reps': _flat, 'length': seq_record["length"]}, ignore_index=True) 
    print(process_id+" Finish")
    output = open(pro_plk, 'wb')
    pickle.dump(ready_df, output)
    output.close()


'''
def convert_amp(df):
    process_id =str(getpid())
    path = "/mnt/vdb/Bat/pws/tranpi/reps"
    pro_plk= path + "chunk"+process_id+".pkl"
    if not os.path.exists(path):
        os.makedirs(path)
    
    print(process_id+" Start")

    char_code_list = [char for char in "ACDEFGHIKLMNPQRSTVWY"] 

    ready_df = pd.DataFrame(columns = ['ID', 'Sequence','reps', 'length'])
    for seq_record in df.iterrows():
        #print(seq_record)
        m = motifs.create([ seq_record[1]["Sequence"] ] , alphabet="ACDEFGHIKLMNPQRSTVWY")  
        index_pattern = [char for char in m.consensus]
        pwm = m.counts.normalize(pseudocounts=0.5)
        pssm_df = pd.DataFrame.from_dict(pwm)
        pssm_df.index = index_pattern
        result_df = pd.DataFrame(0, columns=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'],
        index = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'])
        for  char in char_code_list: # column
            for index, row in pssm_df.iterrows():
                result_df.loc[index, char] += row[char]
        _flat = result_df.values.flatten()
 
        ready_df = ready_df.append({'ID': seq_record[1]["ID"], 'Sequence': seq_record[1]["Sequence"],
                                   'reps': _flat, 'length': seq_record[1]["length"]}, ignore_index=True) 
    print(process_id+" Finish")
    output = open(pro_plk, 'wb')
    pickle.dump(ready_df, output)
    output.close()

'''

def parallelize_dataframe(df, func):
    _tmp_df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    pool.map(func, _tmp_df_split)

    pool.close()
    pool.join()
    
    
def get_df(fastas):
    with open(fastas) as fasta_file:  # Will close handle cleanly
        identifiers = []
        lengths = []
        seqs = []
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
            identifiers.append(seq_record.id)
            # Remove leading and trailing characters from a string
            seqs.append(str(seq_record.seq.strip('*')))
            lengths.append(len(seq_record.seq))
    # dictionary of lists  
    dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
    df = pd.DataFrame(dict) 
    return df

## CD100
before run this, please restart kernel before  and check the  path 

In [3]:
fastas = "/home/ubuntu/data/bk_fasta/SRR12103592.assembly.len10.cd100.fasta"
df = get_df(fastas)
df  

,ID,Sequence,length
0,9,EDTGFYPSEPMLCSESEEGQVPHSLETLYQSADCSSPSDALIVCIH...,275
1,19,ALGPSLWDRRRSLHLLLQEAFPVAQSLAQVIHHQFQTVSKQGGPLP,47
2,32,VGICGSDVHYWQHGRIGDFIVKKPMVLGHEASGTVVKVGSLVKHLQ...,151
3,54,EKPCNSNQQPLENLVEDTLINYSQFGSPKDHEHNGCKLCQTDRYCE...,199
4,61,AAKFVFRHNDPDHLEKLLKKSNSETPKIVAFETVHSMDGAICPLEE...,152
...,...,...,...
422791,12568817,GAWTEVGLPSQDVSVASCNCCRRPMHFELMSEWERSYFGNMGPQYV...,51
422792,12568833,LLLPLKVLGFLGGQLSVVVLHQPVHIIIIQLQAMDLEIFSSFAP,45
422793,12568849,GGQTPRTAPNPQNPPLPPFWAWSSQNVQFSFKFLILGAEPLDRFLA...,117
422794,12568850,VEESCTIENNSDSTKPKMAAEVDFGDLELFEAFDHPEESLPKPVHT...,174


In [4]:
df[df["length"] < 100]

,ID,Sequence,length
1,19,ALGPSLWDRRRSLHLLLQEAFPVAQSLAQVIHHQFQTVSKQGGPLP,47
5,71,RESRPRVPTNARQSSFSLYESGSPETHPELEFRAVGSLKLPPD,44
6,73,VRSLSAVGLFFGFLVSAILTKLWNVDDLKVIIISFSIM,39
7,81,AQKRRWWRWHLSAPWRSSCCRASPAAQQRPAWSPKSPLPTTQP,43
8,82,HRRGDGGAGISRRPGAPPAVGPLLLLSRGLHGAPNHPFLLHNL,44
...,...,...,...
422786,12568410,MSTKSRVPAMEPPTQPKAWPRVPEMMSILCMIPSRSPIPAPREPYSPTA,50
422788,12568523,TNQGQEKNSSNFIEPRRPPETKNRTNDVDFSTSSFSRSKTKTGVGP...,83
422789,12568576,QPVGRDPFGAQTTLSQGSPKTRGPQTFSTGGQFTVPQTVGGPDYSF...,92
422791,12568817,GAWTEVGLPSQDVSVASCNCCRRPMHFELMSEWERSYFGNMGPQYV...,51


In [ ]:
parallelize_dataframe(df, convert_pssm)

In [ ]:
# merge


## Tranpi
before run this, please restart kernel before  and check the  path 

In [4]:
df_tranpi = get_df("/mnt/vdb/Bat/transpi/SRR12103592.combined.okay.fa.transdecoder.pep")
NON_CODE = "B|Z|J|U|O|X"
# remove ambigous seqeunce and non-canonical amino acids codes
df_tranpi = df_tranpi[~df_tranpi["Sequence"].str.contains(NON_CODE, regex=True)]
df_tranpi

,ID,Sequence,length
0,SOAP.k25.C303246.p1,QKVLQAAGPSTTTETETIAKYEIMDGAPVKGESIPIRLFLAGYDPT...,101
1,SOAP.k25.C303708.p1,PITWGRKWNIENGCARTHSQDDYSPGSQAQGESGTASHPRRGHLEM...,102
2,SOAP.k25.C304032.p1,LRNHSPLMSFGASFVSFLNAMMTFEEEKMQLACDDLRTTEKLCESE...,103
3,SOAP.k25.C304284.p1,ESTDQISPYGNSTVTQPSDSGWQYNETHTSLKQNTPRNTSKLYIGL...,104
4,SOAP.k25.C304386.p1,LGPDLSWAWEAKQPWGQETSLRRGEGSGLCKVGGVRVCAPPLLTPK...,104
...,...,...,...
11000,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,MSPSQAVYIVPSKGRLIGGLRDTPSYEHFQEDFSTCSLCTFRDLCA...,106
11001,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,QDLENAATGDAAVHQRIASLPVEVQEVSLLDKITDKESGERLSKMV...,105
11002,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,ARAALAMPVKGGTKCIKYLLLGFNFVFWLAGIAVLAIGLWLRFDSQ...,233
11003,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,MSCKPQCSLNHLPTPCARQSAPFRIPAEFLYLVLLLVEGAPFSNFS...,367


In [5]:
parallelize_dataframe(df_tranpi, convert_pssm)

7924 Start7925 Start
7927 Start7928 Start7926 Start

7930 Start7929 Start7931 Start
7932 Start




7933 Start
7939 Start7938 Start7940 Start

7942 Start
7943 Start7944 Start7946 Start7941 Start7947 Start7945 Start






7937 Start7934 Start
7935 Start

7936 Start
7947 Finish
7925 Finish
7938 Finish
7937 Finish
7940 Finish
7936 Finish
7939 Finish
7924 Finish
7944 Finish
7931 Finish
7935 Finish
7928 Finish
7934 Finish
7933 Finish
7942 Finish
7943 Finish
7941 Finish
7932 Finish
7929 Finish
7927 Finish
7945 Finish
7926 Finish
7930 Finish
7946 Finish


In [5]:
## Test
infile = "/mnt/vdb/Bat/pws/transpi/reps/repschunk2855.pkl"
data = pd.read_pickle(infile)
data

,ID,Sequence,reps,length
0,SOAP.k25.C303246.p1,QKVLQAAGPSTTTETETIAKYEIMDGAPVKGESIPIRLFLAGYDPT...,"[0.5215213479934805, 0.4979208040639721, 0.497...",101
1,SOAP.k25.C303708.p1,PITWGRKWNIENGCARTHSQDDYSPGSQAQGESGTASHPRRGHLEM...,"[0.5286813505675368, 0.4972277511804545, 0.497...",102
2,SOAP.k25.C304032.p1,LRNHSPLMSFGASFVSFLNAMMTFEEEKMQLACDDLRTTEKLCESE...,"[0.535136166304903, 0.49660199406220445, 0.496...",103
3,SOAP.k25.C304284.p1,ESTDQISPYGNSTVTQPSDSGWQYNETHTSLKQNTPRNTSKLYIGL...,"[0.5243796901350509, 0.4976442482006647, 0.497...",104
4,SOAP.k25.C304386.p1,LGPDLSWAWEAKQPWGQETSLRRGEGSGLCKVGGVRVCAPPLLTPK...,"[0.5278557423283019, 0.4973077183277213, 0.497...",104
...,...,...,...,...
420,SPADES.k25.NODE_10859_length_456_cov_7.684455_...,EQVAMECATWLAEQRVPVRVQLKPEISPTQDIRLWVSVEDAQMHTV...,"[0.523988192248591, 0.49768213580912857, 0.497...",151
421,SPADES.k25.NODE_10894_length_455_cov_8.158140_...,SVPHWIEDVRKYAGSNIVQLLIGNKSDLGELREVPLAEAQSLVEHY...,"[0.5241478655622194, 0.49766668360479066, 0.49...",106
422,SPADES.k25.NODE_10900_length_454_cov_112.03030...,MMALASSLCIVCTSFLSSKEKSMISPEKHKKVLFLPISGVFHCLGY...,"[0.525608884899018, 0.4975252727334301, 0.4975...",100
423,SPADES.k25.NODE_10918_length_454_cov_28.289044...,EGHQKHSNIDGMAALHFTAQSNNVRIVEYLTQDLHLKDLDQPDENG...,"[0.544497349359224, 0.49569241426841826, 0.495...",131


In [5]:
data[[ "reps"]]

,reps
0,"[1.6363636363636358, 0.5454545454545455, 0.545..."
1,"[0.5454545454545454, 0.18181818181818182, 0.18..."
2,"[1.227272727272727, 0.40909090909090917, 0.409..."
3,"[0.5454545454545454, 0.18181818181818182, 0.18..."
4,"[1.9090909090909083, 0.6363636363636364, 0.636..."
...,...
17612,"[1.3636363636363633, 0.45454545454545464, 0.45..."
17613,"[0.2727272727272727, 0.09090909090909091, 0.09..."
17614,"[8.590909090909099, 2.863636363636362, 2.86363..."
17615,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
